In [1]:
# Keras

from keras._tf_keras.keras.models import model_from_json
from keras._tf_keras.keras.layers import Dense, Input, Flatten, Dropout, Activation, \
    BatchNormalization, Conv2D, MaxPooling1D, AveragePooling1D
from keras._tf_keras.keras.utils import to_categorical
# from keras._tf_keras.keras.callbacks import ModelCheckpoint
# from keras._tf_keras.keras import utils
from keras._tf_keras.keras import optimizers
# import np_utils

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other
import librosa
from librosa import feature
# import json
# from scipy.io import wavfile
# from python_speech_features import mfcc
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow as tf
# from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
# import glob
from tqdm import tqdm
import os
import pickle
# import IPython.display as ipd  # To play sound in the notebook
#
# import sys
# import warnings

In [5]:

# Load the pre-trained model
json_file = open(r'M:\pythonProject\2D model_json 1.json (slightly improved)', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(r"M:\pythonProject\saved_models\2D Emotion_Model (slightly improved).h5")
print("Loaded model from disk")

# Compile the loaded model
opt = optimizers.Adam(0.001)
loaded_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

# Function to preprocess and extract MFCC features
def extract_mfcc_features2(file_name, n_mfcc=30, sampling_rate=44100, audio_duration=5):
    input_length = sampling_rate * audio_duration
    data, _ = librosa.load(file_name, sr=sampling_rate, res_type="kaiser_fast", duration=5, offset=0.5)

    if len(data) > input_length:
        max_offset = len(data) - input_length
        offset = np.random.randint(max_offset)
        data = data[offset:(input_length + offset)]
    else:
        if input_length > len(data):
            max_offset = input_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")

    feature_mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=n_mfcc)
    feature_mfcc = np.expand_dims(feature_mfcc, axis=-1)
    feature_mfcc = feature_mfcc.reshape(1, n_mfcc, -1, 1)  # Adjust based on your specific model input shape

    return feature_mfcc

# Function to make prediction on a new sound file
def predict_emotion2(file_name):
    label = r'C:\Users\eo19181\PycharmProjects\pythonProject\labeler'
    infile = open(label, 'rb')
    lb = pickle.load(infile)
    infile.close()
    
    features = extract_mfcc_features2(file_name)
    if features is not None:
        prediction = loaded_model.predict(features, batch_size=16, verbose=1)
        predicted_emotion = np.argmax(prediction, axis=1)
        
        predicted_emotion = predicted_emotion.astype(int).flatten()
        predicted_emotion = lb.inverse_transform(predicted_emotion) # Corrected transformation

        return predicted_emotion
    else:
        return None

# Example usage
file_name = r"C:\Users\eo19181\Documents\All -  Datasets\CREMA - Dataset\Male\Disgust\1001_IEO_DIS_HI.wav"
predicted_emotion = predict_emotion2(file_name)
print(f'Predicted Emotion: {predicted_emotion}')

Loaded model from disk


C:\Python3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted Emotion: ['male_disgust']
